In [1]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}[/INST]")

llm = HuggingFaceHub(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
        "max_new_tokens": 250, 
    }
)

chain = prompt | llm

chain.invoke({"word": "potato"})

c:\Users\CWS\Documents\NomadCoder\fullstack-gpt\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'?\n\nPotato is a starchy, tuberous crop from the perennial Solanum tuberosum of the Solanaceae family. The word potato may refer to the plant itself as well as the edible tuber. In the region of the Andes, there are some other closely related cultivated potato species.\n\nWhat is the meaning of potato in English?\n\nPotato is a starchy, tuberous crop from the perennial Solanum tuberosum of the Solanaceae family. The word potato may refer to the plant itself as well as the edible tuber. In the region of the Andes, there are some other closely related cultivated potato species.\n\nWhat is the meaning of potato in Hindi?\n\nPotato is a starchy, tuberous crop from the perennial Solanum tuberosum of the Solanaceae family. The word potato may refer to the plant itself as well as the edible tuber. In the region of the Andes, there are some other closely related cultivated potato species.\n\nWhat is the meaning of potato in Telugu?\n'

Authentication failed for https://api.smith.langchain.com/runs/750c2098-25fb-4164-8728-aa21fb4a6874. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/750c2098-25fb-4164-8728-aa21fb4a6874', '{"detail":"Invalid auth"}')
